# Assignment 3

## Instructions
- Run this notebook on ```Google Colab(preferable)```
- Write your code and analysis in the indicated cells.
- Ensure that this notebook runs without errors when the cells are run in sequence.
- Do not attempt to change the contents of other cells. 

## Packages Used
- sklearn [link](https://scikit-learn.org/)
- Keras [link](https://keras.io/guides/)

## Submission
- Rename the notebook to `<roll_number>_Assignment3_Q3.ipynb`.


## Question 3
Fake news is a widespread problem and there are many methods for combating it.
You have to build a fake news detection system using a ML model. Train any ML model (ANN, LSTM) over the given Dataset.
The dataset has short statements spoken by people and has the meta-information and corresponding label for those sentences. 
Your target is label column which has 6 labels(in the increasing order of truthfullness): pants-fire, false, barely-true, half-true, mostly-true, true.

The features are 'statement', 'subject', 'speaker', 'job', 'state', 'party', 'barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c', 'pants_on_fire_c', 'venue' and the target is column "label".

The statement is made by speaker whose job, party are given along with 6 columns which are an account of the  type of news(labels) the person has shared before. 
The person who has shared fake content before is likely to share it in future and this can be accounted by the ML model as a feature. Column barely_true_c contains how many barely_true news has the speaker shared (and so is with column X_c, value of X_c is number of X the person shared).


You have to perform two tasks:
* task1: Binary classification <br>
Classify the given news as true/false. Take the labels pants-fire, false, barely-true as false and rest (half-true, mostly-true, true) as true.
* task2: Six-way classification <br>
Classify the given news into six-classes "pants-fire, false, barely-true, half-true, mostly-true, true".

For each of the tasks:
1) Experiment with depth of network and try to fine-tune hyperparameters reporting your observations. <br>
2) Report the accuracy, f1-score, confusion matrix on train, val and test sets. <br>
3) Experiment with bag-of-words, glove and bert embeddings(code given in the below notebook) and report results. <br> Comment on what is the affect of embedding on the results.

The pre-processing code is provided, you need to write the training and test.

Note: You are supposed to train on trainset, fine-tune on val and just eval on test set. If found that you trained on val/test sets, the penalty will be incurred.

In [ ]:
# !pip install numpy
# !pip install tensorflow
# !pip install re
# !pip install nltk
# !pip install keras
# !pip install sklearn

In [1]:
# Importing libraries
import numpy as np
import pandas as pd
from tensorflow import keras  #feel free to use any other library
import numpy as np

import re
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from keras.utils import np_utils
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import f1_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# train = pd.read_csv('q3_data/train.csv')
# val = pd.read_csv('q3_data/val.csv')
# test = pd.read_csv('q3_data/test.csv')

train = pd.read_csv('../content/q3_data/train.csv')
val = pd.read_csv('../content/q3_data/val.csv')
test = pd.read_csv('../content/q3_data/test.csv')

In [3]:
# Dropping the 'id' column
train.drop('id', axis = 1, inplace = True)
test.drop('id', axis = 1, inplace = True)
val.drop('id', axis = 1, inplace = True)

In [4]:
train.head(5)

,label,statement,subject,speaker,job,state,party,barely_true_c,false_c,half_true_c,mostly_true_c,pants_on_fire_c,venue
0,False,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0,1,0,0,0,a mailer
1,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0,0,1,1,0,a floor speech.
2,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70,71,160,163,9,Denver
3,False,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7,19,3,5,44,a news release
4,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15,9,20,19,2,an interview on CNN


In [5]:
# Checking the shape of data
print(train.shape)
print(val.shape)
print(test.shape)

(10269, 13)
(1284, 13)
(1283, 13)


## Clean and pre-process data
* Replace missing values
* Remove numbers and special characters
* Convert to upper-case

We experiment with two types of processing, one directly appending the other attributes like subject, job, state, party to sentence and then applying bag of words on it.

Other being encoding sentence with glove embeddings and passing just that.

In [6]:

def dataPreprocessing(data):
    '''Function for cleaning the dataset
    '''
    corpus = []
    # Missing values
    data["job"].fillna("no-job", inplace = True)
    data["state"].fillna("no-state", inplace = True)

    for x in range(data.shape[0]):
        statement = re.sub('[^a-zA-Z]', ' ', data['statement'][x]) # Removing all numbers and special characters
        statement = statement.lower() # Converting uppercase to lowercase
        statement = statement.split()
        
        # you can experiment with any other stemmers
        ps = PorterStemmer()
        statement = [ps.stem(word) for word in statement if not word in set(stopwords.words('english'))] # Stemming the dataset and removing stopwords
        statement = ' '.join(statement)
        subject = data['subject'][x].replace(',', ' ')
        speaker = data['speaker'][x]
        job = data['job'][x].lower()
        # job = job.replace(' ', '-')
        state = data['state'][x].lower()
        party = data['party'][x].lower()
        corpus.append(statement + ' '  + subject + ' ' + job + ' ' + state + ' ' + party)
    return corpus

In [7]:
x_train = dataPreprocessing(train)
x_val = dataPreprocessing(val) 
x_test = dataPreprocessing(test) 

In [8]:
len(x_train), len(x_val), len(x_test)

(10269, 1284, 1283)

In [9]:
corpus = x_train + x_val + x_test

## Using bag-of-words embedding


In [10]:
# Converting the corpus into bag-of-words
cv = CountVectorizer(max_features = 8000)
X = cv.fit_transform(corpus).toarray()

In [11]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [12]:
X.shape

(12836, 8000)

In [13]:
train.columns

Index(['label', 'statement', 'subject', 'speaker', 'job', 'state', 'party',
       'barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c',
       'pants_on_fire_c', 'venue'],
      dtype='object')

In [14]:
# Selecting the columns 'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c'
label_cols = ['barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c',
       'pants_on_fire_c']
x_train2 = train[label_cols]
x_val2 = val[label_cols]
x_test2 = test[label_cols]

In [15]:
# Stacking x_train and x_train2 horizontally
x_train_bow = np.hstack((X[:len(x_train)], x_train2))
x_val_bow = np.hstack((X[len(x_train):len(x_train)+len(x_val)], x_val2))
x_test_bow = np.hstack((X[len(x_train)+len(x_val):], x_test2))

In [16]:
x_train_bow.shape

(10269, 8005)

## Use of Glove Embedding


download glove embeddings from 'https://nlp.stanford.edu/data/glove.6B.zip','glove.6B.zip'
and place in your current working folder


In [17]:
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip "glove.6B.zip" -d "glove"

--2022-04-16 00:57:38--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-04-16 00:57:39--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.05MB/s    in 2m 40s  

2022-04-16 01:00:19 (5.13 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove/glove.6B.50d.txt  
  inflating: glove/glove.6B.100d.txt  
  inflating:

In [18]:
emmbed_dict = {}
with open('glove/glove.6B.200d.txt','r') as f:
  for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:],'float32')
    emmbed_dict[word]=vector


In [19]:
emmbed_dict['oov'] = np.zeros(200)

In [20]:
pip install sentence-transformers

     |████████████████████████████████| 79 kB 3.2 MB/s 
     |████████████████████████████████| 4.0 MB 11.2 MB/s 
     |████████████████████████████████| 1.2 MB 51.6 MB/s 
     |████████████████████████████████| 77 kB 6.6 MB/s 
     |████████████████████████████████| 895 kB 33.8 MB/s 
     |████████████████████████████████| 596 kB 56.0 MB/s 
     |████████████████████████████████| 6.6 MB 39.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=4ea5bb7358506016d3a05946176ead860de6e535b725567930d2b8300ac94533
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [21]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [22]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')
def dataPreprocessing_glove(data):
    corpus = []
    # Missing values
    data["job"].fillna("no-job", inplace = True)
    data["state"].fillna("no-state", inplace = True)

    for x in range(data.shape[0]):
        statement = re.sub('[^a-zA-Z]', ' ', data['statement'][x]) # Removing all numbers and special characters
        statement = statement.lower() # Converting uppercase to lowercase
        statement = word_tokenize(statement)

        embed_statement = []
        for w in statement:
            if w in emmbed_dict:
                embed_statement.append(emmbed_dict[w])
            else:
                embed_statement.append(emmbed_dict['oov'])
         
        # bonus: Think how you can encode the below features(hint: look upon label encoding or training your own word2vec or any other embedding model)
    
#         subject = data['subject'][x].replace(',', ' ')
#         speaker = data['speaker'][x]
#         job = data['job'][x].lower()
#         # job = job.replace(' ', '-')
#         state = data['state'][x].lower()
#         party = data['party'][x].lower()
        corpus.append(embed_statement)
    
    return np.array(corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [23]:
x_train_glove = dataPreprocessing_glove(train)
x_val_glove = dataPreprocessing_glove(val) 
x_test_glove = dataPreprocessing_glove(test) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [24]:
x_train_glove = np.hstack((x_train_glove.reshape(-1,1), x_train2))
x_val_glove = np.hstack((x_val_glove.reshape(-1,1), x_val2))
x_test_glove = np.hstack((x_test_glove.reshape(-1,1), x_test2))

In [25]:
max_len = 0
for i in range(x_train_glove.shape[0]):
    max_len  = max(max_len, len(x_train_glove[i][0]))
max_len

65

In [26]:
x_train_glove_new = np.zeros((x_train_glove.shape[0], 66, 200))
x_val_glove_new = np.zeros((x_val_glove.shape[0], 66, 200))
x_test_glove_new = np.zeros((x_test_glove.shape[0], 66, 200))

In [27]:
for i in range(x_train_glove.shape[0]):
    embedding = np.array(x_train_glove[i][0])
    x_train_glove_new[i,0:embedding.shape[0],:] = embedding

In [28]:
for i in range(x_val_glove.shape[0]):
    embedding = np.array(x_val_glove[i][0])
    x_val_glove_new[i,0:embedding.shape[0],:] = embedding

In [29]:
for i in range(x_test_glove.shape[0]):
    embedding = np.array(x_test_glove[i][0])
    x_test_glove_new[i,0:embedding.shape[0],:] = embedding

In [30]:
len(x_train_glove_new[69][0])

200

## Use of bert embeddings
note: we used our pre-processed code for bow which has the attributed appended to end the end of sentence. 

In [31]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

x_train_bert = np.hstack((model.encode(x_train), x_train2))
x_val_bert = np.hstack((model.encode(x_val), x_val2))
x_test_bert = np.hstack((model.encode(x_test), x_test2))

In [32]:
x_val_bert.shape

(1284, 389)

Now use the above 3 types of embedded inputs(bow, glove, bert embeddings) for the 2 classification tasks and compare their outputs


# Six-way classification

## Preprocessing

In [33]:
num_classes = 6
# Preprocessing function for the labels
def categorize(data):
    y = data["label"].tolist()

    # Encoding the Dependent Variable
    labelencoder_y = LabelEncoder()
    y = labelencoder_y.fit_transform(y)

    # Converting to binary class matrix
    y = np_utils.to_categorical(y, num_classes)
    return y

In [34]:
y_train_six_way = categorize(train)
y_test_six_way = categorize(test)
y_val_six_way = categorize(val)

Build a model and pass bow, glove and bert embedded inputs: x_train_bow, x_train_glove, x_train_bert(similarly validate for val and report results on test)


## Model

In [35]:
x_train_bow.shape

(10269, 8005)

In [36]:
y_test_six_way.shape

(1283, 6)

In [37]:
def model_eval(x_test, y_test, model, no_of_labels):

    y_pred = np.zeros((y_test.shape[0],no_of_labels))
    scores = model.evaluate(x_test, y_test)
    pred = model.predict(x_test)
    for i in range(y_test.shape[0]):
        index = np.argmax(pred[i])
        y_pred[index] = 1 
    print("Testing Accuracy : ", scores[1])
    
    print("Confusion Matrix : ")
    cv = multilabel_confusion_matrix(y_test, y_pred)
    print(cv)
    
    print("F1 scores :", f1_score(y_test, y_pred, average = None))
    

### Bag of Words Embedding

In [38]:
model_bow_6 = Sequential()
model_bow_6.add(Dense(1000, activation='relu'))
model_bow_6.add(Dropout(0.75))
model_bow_6.add(Dense(200, activation='relu'))
model_bow_6.add(Dropout(0.75))
model_bow_6.add(Dense(6,activation='softmax'))
model_bow_6.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model_bow_6.fit(x_train_bow, y_train_six_way , validation_data = (x_val_bow, y_val_six_way), epochs=10)

Epoch 1/10
321/321 [==============================] - 20s 59ms/step - loss: 2.0233 - accuracy: 0.2324 - val_loss: 1.6513 - val_accuracy: 0.3302
Epoch 2/10
321/321 [==============================] - 19s 58ms/step - loss: 1.6642 - accuracy: 0.3199 - val_loss: 1.5345 - val_accuracy: 0.3801
Epoch 3/10
321/321 [==============================] - 19s 58ms/step - loss: 1.5139 - accuracy: 0.3820 - val_loss: 1.4851 - val_accuracy: 0.3894
Epoch 4/10
321/321 [==============================] - 19s 58ms/step - loss: 1.4052 - accuracy: 0.4225 - val_loss: 1.4431 - val_accuracy: 0.3980
Epoch 5/10
321/321 [==============================] - 19s 58ms/step - loss: 1.3189 - accuracy: 0.4605 - val_loss: 1.4312 - val_accuracy: 0.4019
Epoch 6/10
321/321 [==============================] - 19s 58ms/step - loss: 1.2443 - accuracy: 0.4853 - val_loss: 1.4603 - val_accuracy: 0.3956
Epoch 7/10
321/321 [==============================] - 19s 60ms/step - loss: 1.1969 - accuracy: 0.5040 - val_loss: 1.4653 - val_accuracy:

In [39]:
model_eval(x_test_bow, y_test_six_way,model_bow_6,6)

41/41 [==============================] - 1s 21ms/step - loss: 1.5680 - accuracy: 0.3819
Testing Accuracy :  0.38191738724708557
Confusion Matrix : 
[[[1029    4]
  [ 248    2]]

 [[1068    4]
  [ 209    2]]

 [[1063    6]
  [ 214    0]]

 [[1011    5]
  [ 266    1]]

 [[1028    6]
  [ 249    0]]

 [[1186    5]
  [  91    1]]]
F1 scores : [0.015625   0.01843318 0.         0.00732601 0.         0.02040816]


### Glove Embedding

In [40]:
model_glove_6 = Sequential()
model_glove_6.add(LSTM(400))
model_glove_6.add(Dense(100, activation='relu'))
model_glove_6.add(Dropout(0.2))
model_glove_6.add(Dense(6,activation='softmax'))
model_glove_6.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model_glove_6.fit(x_train_glove_new, y_train_six_way , validation_data = (x_val_glove_new, y_val_six_way), epochs=10)

Epoch 1/10
321/321 [==============================] - 192s 590ms/step - loss: 1.7623 - accuracy: 0.2001 - val_loss: 1.7649 - val_accuracy: 0.1931
Epoch 2/10
321/321 [==============================] - 160s 500ms/step - loss: 1.7585 - accuracy: 0.2003 - val_loss: 1.7656 - val_accuracy: 0.1924
Epoch 3/10
321/321 [==============================] - 159s 496ms/step - loss: 1.7593 - accuracy: 0.1980 - val_loss: 1.7654 - val_accuracy: 0.1924
Epoch 4/10
321/321 [==============================] - 162s 506ms/step - loss: 1.7582 - accuracy: 0.1984 - val_loss: 1.7652 - val_accuracy: 0.1900
Epoch 5/10
321/321 [==============================] - 179s 557ms/step - loss: 1.7592 - accuracy: 0.2062 - val_loss: 1.7651 - val_accuracy: 0.1924
Epoch 6/10
321/321 [==============================] - 181s 562ms/step - loss: 1.7570 - accuracy: 0.2050 - val_loss: 1.7644 - val_accuracy: 0.1963
Epoch 7/10
321/321 [==============================] - 174s 542ms/step - loss: 1.7578 - accuracy: 0.2079 - val_loss: 1.7627 -

In [41]:
model_eval(x_test_glove_new, y_test_six_way,model_glove_6,6)

41/41 [==============================] - 6s 152ms/step - loss: 1.7489 - accuracy: 0.2081
Testing Accuracy :  0.20810599625110626
Confusion Matrix : 
[[[1032    1]
  [ 250    0]]

 [[1071    1]
  [ 211    0]]

 [[1068    1]
  [ 214    0]]

 [[1016    0]
  [ 266    1]]

 [[1033    1]
  [ 249    0]]

 [[1190    1]
  [  92    0]]]
F1 scores : [0.         0.         0.         0.00746269 0.         0.        ]


### BERT Embedding

In [42]:
model_bert_6 = Sequential()
model_bert_6.add(Dense(500, activation='relu'))
model_bert_6.add(Dropout(0.75))
model_bert_6.add(Dense(6,activation='softmax'))
model_bert_6.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model_bert_6.fit(x_train_bert, y_train_six_way , validation_data = (x_val_bert, y_val_six_way), epochs=10)

Epoch 1/10
321/321 [==============================] - 2s 5ms/step - loss: 2.5971 - accuracy: 0.2554 - val_loss: 1.5996 - val_accuracy: 0.3746
Epoch 2/10
321/321 [==============================] - 2s 5ms/step - loss: 1.6488 - accuracy: 0.3288 - val_loss: 1.5285 - val_accuracy: 0.3871
Epoch 3/10
321/321 [==============================] - 1s 5ms/step - loss: 1.5469 - accuracy: 0.3654 - val_loss: 1.4686 - val_accuracy: 0.4159
Epoch 4/10
321/321 [==============================] - 1s 4ms/step - loss: 1.4990 - accuracy: 0.3814 - val_loss: 1.4459 - val_accuracy: 0.4338
Epoch 5/10
321/321 [==============================] - 1s 4ms/step - loss: 1.4539 - accuracy: 0.4047 - val_loss: 1.4044 - val_accuracy: 0.4354
Epoch 6/10
321/321 [==============================] - 1s 5ms/step - loss: 1.4120 - accuracy: 0.4191 - val_loss: 1.3746 - val_accuracy: 0.4393
Epoch 7/10
321/321 [==============================] - 2s 5ms/step - loss: 1.3860 - accuracy: 0.4314 - val_loss: 1.3535 - val_accuracy: 0.4525
Epoch 

In [43]:
model_eval(x_test_bert, y_test_six_way,model_bert_6,6)

41/41 [==============================] - 0s 2ms/step - loss: 1.3415 - accuracy: 0.4552
Testing Accuracy :  0.4551831781864166
Confusion Matrix : 
[[[1029    4]
  [ 248    2]]

 [[1068    4]
  [ 209    2]]

 [[1063    6]
  [ 214    0]]

 [[1011    5]
  [ 266    1]]

 [[1028    6]
  [ 249    0]]

 [[1186    5]
  [  91    1]]]
F1 scores : [0.015625   0.01843318 0.         0.00732601 0.         0.02040816]


# Binary Classification

## Preprocessing

In [44]:
num_classes = 2

In [45]:
# Function for preprocessing labels
def dataPreprocessingBinary(data):
    y = data["label"].tolist()

    # Changing the 'half-true', 'mostly-true', barely-true', 'pants-fire' labels to True/False for Binary Classification
    for x in range(len(y)):
        if(y[x] == 'half-true'):
            y[x] = 'True'
        elif(y[x] == 'mostly-true'):
            y[x] = 'True'
        elif(y[x] == 'barely-true'):
            y[x] = 'False'
        elif(y[x] == 'pants-fire'):
            y[x] = 'False'

    # Converting the lables into binary class matrix
    labelencoder_y = LabelEncoder()
    y = labelencoder_y.fit_transform(y)
    y = np_utils.to_categorical(y, num_classes)
    return y

In [46]:
y_train_binary = dataPreprocessingBinary(train)
y_test_binary = dataPreprocessingBinary(test)
y_val_binary = dataPreprocessingBinary(val)

## Model
Build a model and pass bow, glove and bert embedded inputs: x_train_bow, x_train_glove, x_train_bert(similarly validate for val and report results on test)


### Bag of Words Embedding

In [47]:
## write your code here
# Initialize hyperparameters
# Create model
# train
# test
# report accuracy, f1-score and confusion matrix

model_bow_2 = Sequential()
model_bow_2.add(Dense(1000, activation='relu'))
model_bow_2.add(Dropout(0.75))
model_bow_2.add(Dense(200, activation='relu'))
model_bow_2.add(Dropout(0.75))
model_bow_2.add(Dense(2,activation='softmax'))
model_bow_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_bow_2.fit(x_train_bow, y_train_binary , validation_data = (x_val_bow, y_val_binary), epochs=10)

Epoch 1/10
321/321 [==============================] - 21s 63ms/step - loss: 0.7518 - accuracy: 0.6247 - val_loss: 0.5852 - val_accuracy: 0.6893
Epoch 2/10
321/321 [==============================] - 20s 61ms/step - loss: 0.5934 - accuracy: 0.6934 - val_loss: 0.5784 - val_accuracy: 0.6791
Epoch 3/10
321/321 [==============================] - 20s 61ms/step - loss: 0.5330 - accuracy: 0.7242 - val_loss: 0.5694 - val_accuracy: 0.6900
Epoch 4/10
321/321 [==============================] - 20s 61ms/step - loss: 0.4900 - accuracy: 0.7469 - val_loss: 0.5815 - val_accuracy: 0.6924
Epoch 5/10
321/321 [==============================] - 19s 60ms/step - loss: 0.4532 - accuracy: 0.7695 - val_loss: 0.6178 - val_accuracy: 0.6799
Epoch 6/10
321/321 [==============================] - 19s 60ms/step - loss: 0.4206 - accuracy: 0.7830 - val_loss: 0.6399 - val_accuracy: 0.6783
Epoch 7/10
321/321 [==============================] - 19s 60ms/step - loss: 0.3953 - accuracy: 0.7898 - val_loss: 0.6970 - val_accuracy:

In [48]:
model_eval(x_test_bow, y_test_binary,model_bow_2,2)

41/41 [==============================] - 1s 19ms/step - loss: 0.8311 - accuracy: 0.6781
Testing Accuracy :  0.678098201751709
Confusion Matrix : 
[[[726   1]
  [555   1]]

 [[555   1]
  [726   1]]]
F1 scores : [0.00358423 0.00274348]


### Glove Embedding

In [49]:
model_glove_2 = Sequential()
model_glove_2.add(LSTM(200))
model_glove_2.add(Dense(2,activation='sigmoid'))
model_glove_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_glove_2.fit(x_train_glove_new, y_train_binary , validation_data = (x_val_glove_new, y_val_binary), epochs=10)

Epoch 1/10
321/321 [==============================] - 64s 186ms/step - loss: 0.6861 - accuracy: 0.5613 - val_loss: 0.6964 - val_accuracy: 0.5202
Epoch 2/10
321/321 [==============================] - 59s 183ms/step - loss: 0.6858 - accuracy: 0.5621 - val_loss: 0.6951 - val_accuracy: 0.5202
Epoch 3/10
321/321 [==============================] - 60s 186ms/step - loss: 0.6858 - accuracy: 0.5623 - val_loss: 0.6947 - val_accuracy: 0.5202
Epoch 4/10
321/321 [==============================] - 61s 191ms/step - loss: 0.6854 - accuracy: 0.5622 - val_loss: 0.7007 - val_accuracy: 0.5202
Epoch 5/10
321/321 [==============================] - 62s 193ms/step - loss: 0.6854 - accuracy: 0.5620 - val_loss: 0.6937 - val_accuracy: 0.5202
Epoch 6/10
321/321 [==============================] - 64s 201ms/step - loss: 0.6874 - accuracy: 0.5588 - val_loss: 0.6943 - val_accuracy: 0.5202
Epoch 7/10
321/321 [==============================] - 63s 197ms/step - loss: 0.6857 - accuracy: 0.5607 - val_loss: 0.6928 - val_ac

In [50]:
model_eval(x_test_glove_new, y_test_binary,model_glove_2,2)

41/41 [==============================] - 3s 64ms/step - loss: 0.6616 - accuracy: 0.5900
Testing Accuracy :  0.590023398399353
Confusion Matrix : 
[[[726   1]
  [555   1]]

 [[555   1]
  [726   1]]]
F1 scores : [0.00358423 0.00274348]


### BERT Embedding

In [51]:
model_bert_2 = Sequential()
model_bert_2.add(Dense(500, activation='relu'))
model_bert_2.add(Dropout(0.75))
model_bert_2.add(Dense(2,activation='sigmoid'))
model_bert_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_bert_2.fit(x_train_bert, y_train_binary , validation_data = (x_val_bert, y_val_binary), epochs=10)

Epoch 1/10
321/321 [==============================] - 2s 4ms/step - loss: 0.9184 - accuracy: 0.6316 - val_loss: 0.6193 - val_accuracy: 0.6768
Epoch 2/10
321/321 [==============================] - 1s 4ms/step - loss: 0.6243 - accuracy: 0.6702 - val_loss: 0.5922 - val_accuracy: 0.6760
Epoch 3/10
321/321 [==============================] - 1s 4ms/step - loss: 0.5937 - accuracy: 0.6945 - val_loss: 0.5694 - val_accuracy: 0.6900
Epoch 4/10
321/321 [==============================] - 1s 4ms/step - loss: 0.5700 - accuracy: 0.7044 - val_loss: 0.5762 - val_accuracy: 0.6893
Epoch 5/10
321/321 [==============================] - 1s 4ms/step - loss: 0.5597 - accuracy: 0.7129 - val_loss: 0.5480 - val_accuracy: 0.7040
Epoch 6/10
321/321 [==============================] - 1s 4ms/step - loss: 0.5446 - accuracy: 0.7195 - val_loss: 0.5479 - val_accuracy: 0.6846
Epoch 7/10
321/321 [==============================] - 1s 4ms/step - loss: 0.5333 - accuracy: 0.7223 - val_loss: 0.5354 - val_accuracy: 0.7196
Epoch 

In [52]:
model_eval(x_test_bert, y_test_binary,model_bert_2,2)

41/41 [==============================] - 0s 2ms/step - loss: 0.5313 - accuracy: 0.7334
Testing Accuracy :  0.7334372401237488
Confusion Matrix : 
[[[726   1]
  [555   1]]

 [[555   1]
  [726   1]]]
F1 scores : [0.00358423 0.00274348]
